In [42]:
import requests

resp = requests.get('https://indiapicororno.herokuapp.com/statesstatus')
if resp.status_code != 200:
    # This means something went wrong.
    raise ApiError('GET /tasks/ {}'.format(resp.status_code))

In [48]:
response_dict = resp.json()
response_df = pd.DataFrame.from_records(response_dict)
response_df.head()

,0,1,2,3,4,5
0,1,Andhra Pradesh,3,0,0,0
1,2,Chhattisgarh,1,0,0,0
2,3,Delhi,16,1,5,1
3,4,Gujarat,5,0,0,0
4,5,Haryana,3,14,0,0


In [47]:
indian_states_df = pd.read_csv('../data/Indian_States.csv')
indian_states_df

,State,lat,lon
0,Rajasthan,27.023800,74.217900
1,Kerala,10.850500,76.271100
2,Maharashtra,19.751500,75.713900
3,Goa,15.299300,74.124000
4,Union Territory of Jammu and Kashmir,33.778200,76.576200
5,Karnataka,15.317300,75.713900
6,Gujarat,22.258700,71.192400
7,Haryana,29.058800,76.085600
8,Uttar Pradesh,26.846700,80.946200
9,Bihar,25.096100,85.313100


In [66]:
def read_state_info(row):
    state = row['State']
    target_row = response_df[response_df[1] == state]
    if not target_row.empty:
        row['TotalConfirmedCasesInIndia'] = int(target_row[2].values[0])
        row['TotalConfirmedCasesForeignNational'] = int(target_row[3].values[0])
        row['Cured'] = int(target_row[4].values[0])
        row['Death'] = int(target_row[5].values[0])
    else:
        row['TotalConfirmedCasesInIndia'] = 0
        row['TotalConfirmedCasesForeignNational'] = 0
        row['Cured'] = 0
        row['Death'] = 0
    
    return row

indian_states_corona_df = indian_states_df.apply(lambda x: read_state_info(x), axis=1)
indian_states_corona_df = indian_states_corona_df.sort_values('TotalConfirmedCasesInIndia', ascending=False)
indian_states_corona_df

,State,lat,lon,TotalConfirmedCasesInIndia,TotalConfirmedCasesForeignNational,Cured,Death
2,Maharashtra,19.751500,75.713900,49,3,0,1
1,Kerala,10.850500,76.271100,26,2,3,0
8,Uttar Pradesh,26.846700,80.946200,22,1,9,0
0,Rajasthan,27.023800,74.217900,15,2,3,0
5,Karnataka,15.317300,75.713900,15,0,1,1
30,Union Territory of Ladakh,34.209515,77.615112,10,0,0,0
6,Gujarat,22.258700,71.192400,5,0,0,0
4,Union Territory of Jammu and Kashmir,33.778200,76.576200,4,0,0,0
13,Tamil Nadu,11.127100,78.656900,3,0,1,0
19,Uttarakhand,30.066800,79.019300,3,0,0,0


In [67]:
import pandas as pd

import plotly.express as px

fig = px.scatter_mapbox(indian_states_corona_df, lat="lat", lon="lon", hover_name="State", hover_data=["TotalConfirmedCasesInIndia", "TotalConfirmedCasesForeignNational", "Cured", "Death"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [69]:
import plotly.express as px
fig = px.scatter_geo(indian_states_corona_df, lat='lat', lon='lon', color="State",
                     hover_name="State", size="TotalConfirmedCasesInIndia", locationmode='ISO-3',
                     projection="natural earth")
fig.show()

In [65]:
indian_states_corona_df = indian_states_corona_df.sort_values('TotalConfirmedCasesInIndia', ascending=False)

,State,lat,lon,TotalConfirmedCasesInIndia,TotalConfirmedCasesForeignNational,Cured,Death
2,Maharashtra,19.751500,75.713900,49,3,0,1
1,Kerala,10.850500,76.271100,26,2,3,0
8,Uttar Pradesh,26.846700,80.946200,22,1,9,0
0,Rajasthan,27.023800,74.217900,15,2,3,0
5,Karnataka,15.317300,75.713900,15,0,1,1
30,Union Territory of Ladakh,34.209515,77.615112,10,0,0,0
6,Gujarat,22.258700,71.192400,5,0,0,0
4,Union Territory of Jammu and Kashmir,33.778200,76.576200,4,0,0,0
13,Tamil Nadu,11.127100,78.656900,3,0,1,0
19,Uttarakhand,30.066800,79.019300,3,0,0,0
